In [1]:
import sys, os
import pandas as pd
import setting

import sys, os
import pandas as pd
import setting
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.common.common import p2_dataSelection as p2
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.ML.common import model_manager

import torch

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 2. Training 

## 2-1. Data selection

In [2]:
dataRoot = 'ml_data_integration'
# 1 (p2부분 1. Data Selection)
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['IntegraionTrainX', 'IntegraionTrainy', 'IntegraionTestX', 'IntegraionTesty']

In [4]:
#2
# dataX
dataName_X = dataList[0]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[1]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

#3
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)
dataX = p2.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)
datay = p2.get_saved_integrated_data(dataSaveMode_y, dataName_y, dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

## 2-2. Training Data Preparation

In [5]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)

# 2-2
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam='scale'
scaleMethod='minmax'

# 2-4
splitRatio = 0.8

# 2-5
scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
train_x, val_x, X_scalerFilePath = p3.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
train_y, val_y, y_scalerFilePath = p3.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


## 2-3 Training 

In [6]:
# 3-1.
model_list = ["LSTM_rg","CNN_1D_rg","GRU_rg", "LSTM_FCNs_rg"]
model_method = model_list[0]

#import main_regression as mr

n_epochs = 10 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 16  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)

# TODO: 어디로?
from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
temp_train_x, _ = transDFtoNP(train_x, train_y, windowNum=0)
input_size, seq_len = temp_train_x.shape[1], temp_train_x.shape[2] # input size 크기, seqeunce 길이

trainParameter = setting.modelTestconfig[model_method]
trainParameter['device']  = device
trainParameter['n_epochs'] = n_epochs
trainParameter['batch_size'] = batch_size

trainParameter['input_size'] = input_size
trainParameter['seq_len'] = seq_len

modelTags =["aaaaa"]
trainDataType = "timeseries"
modelPurpose = "regression"

# 2
trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3. 모델을 저장할 파일 패스를 생성한다.

model_name = 'EnergyLSTMRegression'
from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
# trainDataPathList = ["CLUST","Electronics", dataName_X, trainParameter_encode]
trainDataPathList = [model_name, dataName_X, trainParameter_encode]
modelFilePath = model_manager.get_model_file_path(trainDataPathList, model_method)

./Models/LSTM_FCNs_rg/EnergyLSTMRegression/IntegraionTrainX/d85c691b5e33d0aecce784cdafef77b7/model.pkl


/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]


In [7]:
# 4. Training
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML
rml = RML()
rml.set_param(trainParameter)
print(trainParameter)
rml.set_model(model_method)
rml.set_data(train_x, train_y, val_x, val_y)
rml.train()
rml.save_best_model("./yk_LSTM_rg_test.pkl")

{'num_layers': 2, 'lstm_drop_out': 0.4, 'fc_drop_out': 0.1, 'lr': 0.0001, 'device': 'cpu', 'n_epochs': 10, 'batch_size': 16, 'input_size': 24, 'seq_len': 144}
Start training model

Epoch 1/10


/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
/Users/kim-youngkee/Documents/CLUSTER/Clust/examples/7. ModelTest/7-3. Regression/../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError

train Loss: 0.7627
val Loss: 0.3436

Epoch 10/10
train Loss: 0.3865
val Loss: 0.2940

Training complete in 0m 8s
Best val MSE: 0.271234
